In [1]:
import numpy as np
import matplotlib.pyplot as plt
#import missingno as msno
import os
import sys
import seaborn as sns
import pandas as pd
sys.path.insert(0, '../../src/')
from data_loader import load_data, load_game_data

In [3]:
#seasons = load_data(seasons=["2003-04", "2018-19"])
seasons= load_data()
#seasons.head(20)

In [4]:
def create_bins(row, colName):
    if row[colName]==-1:
        return np.nan
    elif row[colName]>=0 and row[colName]<5:
        return '[0-5)'
    elif row[colName]>=5 and row[colName]<10:
        return '[5-10)'
    elif row[colName]>=10 and row[colName]<15:
        return '[10-15)'
    elif row[colName]>=15 and row[colName]<20:
        return '[15-20)' 
    elif row[colName]>=20 and row[colName]<25:
        return '[20-25)'     
    elif row[colName]>=25 and row[colName]<30:
        return '[25-30)'     
    elif row[colName]>=30 and row[colName]<35:
        return '[30-35)'     
    elif row[colName]>=35 and row[colName]<40:
        return '[35-40)'   
    elif row[colName]>=40 and row[colName]<47:
        return '[40-47)'   
    elif row[colName]>=47.0:
        return '[47+)' 
    
seasons['home_shot_distance_bins']=seasons.apply(lambda row: create_bins(row,'home_shot_distance'),axis=1)
seasons['visitor_shot_distance_bins']=seasons.apply(lambda row: create_bins(row,'visitor_shot_distance'),axis=1)
seasons[['HOMEDESCRIPTION', 'VISITORDESCRIPTION', "home_shot_distance","visitor_shot_distance", "home_shot_distance_bins","visitor_shot_distance_bins", "EVENTMSGTYPE"]].head(20)

,HOMEDESCRIPTION,VISITORDESCRIPTION,home_shot_distance,visitor_shot_distance,home_shot_distance_bins,visitor_shot_distance_bins,EVENTMSGTYPE
0,NaN,NaN,-1,-1,NaN,NaN,PERIOD_BEGIN
1,Jump Ball Grant vs. Mutombo: Tip to Hill,NaN,-1,-1,NaN,NaN,JUMP_BALL
2,NaN,Iverson 21' Jump Shot (2 PTS),-1,21,NaN,[20-25),FIELD_GOAL_MADE
3,NaN,Hill P.FOUL (P1.T1),-1,-1,NaN,NaN,FOUL
4,Mason 17' Jump Shot (2 PTS),NaN,17,-1,[15-20),NaN,FIELD_GOAL_MADE
5,NaN,MISS Lynch 22' Jump Shot,-1,22,NaN,[20-25),FIELD_GOAL_MISSED
6,Mason REBOUND (Off:0 Def:1),NaN,-1,-1,NaN,NaN,REBOUND
7,MISS Majerle Driving Layup,NaN,-1,-1,NaN,NaN,FIELD_GOAL_MISSED
8,NaN,Snow REBOUND (Off:0 Def:1),-1,-1,NaN,NaN,REBOUND
9,NaN,Lynch Layup (2 PTS) (Snow 1 AST),-1,-1,NaN,NaN,FIELD_GOAL_MADE


In [5]:
successful = seasons.loc[(((~seasons["home_shot_distance"].equals(-1)) & seasons['EVENTMSGTYPE'].str.contains('FIELD_GOAL_MADE', na=False)) | ((~seasons["visitor_shot_distance"].equals(-1)) & seasons['EVENTMSGTYPE'].str.contains('FIELD_GOAL_MADE', na=False)))]
missed = seasons.loc[(((~seasons["home_shot_distance"].equals(-1)) & seasons['EVENTMSGTYPE'].str.contains('FIELD_GOAL_MISSED', na=False)) | ((~seasons["visitor_shot_distance"].equals(-1)) & seasons['EVENTMSGTYPE'].str.contains('FIELD_GOAL_MISSED', na=False)))]

In [34]:
def get_percentage(bin_type):
    count_of_bin_per_season = seasons[(seasons["home_shot_distance_bins"]==bin_type) | (seasons["visitor_shot_distance_bins"]==bin_type)].groupby("season_name")["EVENTMSGTYPE"].count()
    total_shots_per_season = seasons[(seasons["EVENTMSGTYPE"]=="FIELD_GOAL_MISSED") | (seasons["EVENTMSGTYPE"]=="FIELD_GOAL_MADE")].groupby("season_name")["EVENTMSGTYPE"].count()

    return count_of_bin_per_season

In [58]:
#seasons[(seasons["EVENTMSGTYPE"]=="FIELD_GOAL_MISSED") | (seasons["EVENTMSGTYPE"]=="FIELD_GOAL_MADE")]
missing_distance_info = seasons[((seasons["home_shot_distance_bins"].isnull()) & (seasons["visitor_shot_distance_bins"].isnull())) &((seasons["EVENTMSGTYPE"]=="FIELD_GOAL_MISSED") | (seasons["EVENTMSGTYPE"]=="FIELD_GOAL_MADE"))].groupby("season_name")['EVENTMSGTYPE'].count().to_frame()
#.groupby("season_name")['GAME_ID'].nunique()

In [62]:
seasons[((seasons["home_shot_distance_bins"].isnull()) & (seasons["visitor_shot_distance_bins"].isnull())) &((seasons["EVENTMSGTYPE"]=="FIELD_GOAL_MISSED") | (seasons["EVENTMSGTYPE"]=="FIELD_GOAL_MADE"))]

,Unnamed: 0,EVENTMSGACTIONTYPE,EVENTMSGTYPE,EVENTNUM,GAME_ID,HOMEDESCRIPTION,PCTIMESTRING,PERIOD,PERSON1TYPE,PERSON2TYPE,...,PLAYER3_TEAM_NICKNAME,SCORE,SCOREMARGIN,VISITORDESCRIPTION,WCTIMESTRING,season_name,home_shot_distance,visitor_shot_distance,home_shot_distance_bins,visitor_shot_distance_bins
7,7,Driving Layup,FIELD_GOAL_MISSED,7,20001116,MISS Majerle Driving Layup,10:49,1,HOME_PLAYER,0,...,NaN,NaN,NaN,NaN,12:14 PM,2000-01,-1,-1,NaN,NaN
9,9,Layup,FIELD_GOAL_MADE,9,20001116,NaN,10:44,1,VISITOR_PLAYER,VISITOR_PLAYER,...,NaN,4 - 2,-2,Lynch Layup (2 PTS) (Snow 1 AST),12:14 PM,2000-01,-1,-1,NaN,NaN
19,19,Driving Layup,FIELD_GOAL_MADE,19,20001116,NaN,9:45,1,VISITOR_PLAYER,0,...,NaN,6 - 2,-4,Iverson Driving Layup (4 PTS),12:15 PM,2000-01,-1,-1,NaN,NaN
39,39,Driving Layup,FIELD_GOAL_MISSED,40,20001116,NaN,6:32,1,VISITOR_PLAYER,0,...,NaN,NaN,NaN,MISS Hill Driving Layup,12:20 PM,2000-01,-1,-1,NaN,NaN
44,44,Tip Shot,FIELD_GOAL_MADE,44,20001116,NaN,5:49,1,VISITOR_PLAYER,0,...,NaN,13 - 8,-5,Jones Tip Shot (2 PTS),12:21 PM,2000-01,-1,-1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10389433,114,Step Back Jump Shot,FIELD_GOAL_MISSED,168,21801037,NaN,11:25,2,VISITOR_PLAYER,0,...,NaN,NaN,NaN,MISS Stauskas 3PT Step Back Jump Shot,9:10 PM,2018-19,-1,-1,NaN,NaN
10389558,239,Jump Shot,FIELD_GOAL_MADE,350,21801037,NaN,7:09,3,VISITOR_PLAYER,VISITOR_PLAYER,...,NaN,74 - 79,5,Osman 3PT Jump Shot (12 PTS) (Sexton 2 AST),10:01 PM,2018-19,-1,-1,NaN,NaN
10389559,240,Jump Shot,FIELD_GOAL_MADE,352,21801037,Hardaway Jr. 3PT Jump Shot (22 PTS) (Finney-S...,6:48,3,HOME_PLAYER,HOME_PLAYER,...,NaN,74 - 82,8,NaN,10:01 PM,2018-19,-1,-1,NaN,NaN
10389589,270,Jump Shot,FIELD_GOAL_MISSED,397,21801037,NaN,4:11,3,VISITOR_PLAYER,0,...,NaN,NaN,NaN,MISS Clarkson 3PT Jump Shot,10:06 PM,2018-19,-1,-1,NaN,NaN


In [92]:
missing_distance_info = seasons[((seasons["home_shot_distance_bins"].isnull()) & (seasons["visitor_shot_distance_bins"].isnull())) &((seasons["EVENTMSGTYPE"]=="FIELD_GOAL_MISSED") | (seasons["EVENTMSGTYPE"]=="FIELD_GOAL_MADE"))].groupby("season_name")['EVENTMSGTYPE'].count().to_frame()
#missing_distance_info = missing_distance_info.rename(columns = {'EVENTMSGTYPE':'missing'}, inplace = True)

missing_distance_info["total"] = seasons[(seasons["EVENTMSGTYPE"]=="FIELD_GOAL_MISSED") | (seasons["EVENTMSGTYPE"]=="FIELD_GOAL_MADE")].groupby("season_name")["EVENTMSGTYPE"].count()
missing_distance_info["Missing"] = missing_distance_info["EVENTMSGTYPE"]/  missing_distance_info["total"]
#
missing_distance_info["Present"] = (missing_distance_info["total"] - missing_distance_info["EVENTMSGTYPE"]) /  missing_distance_info["total"]
missing_distance_info

,EVENTMSGTYPE,total,Missing,Present
season_name,,,,
2000-01,47166,191664,0.246087,0.753913
2001-02,51969,193263,0.268903,0.731097
2002-03,52722,192109,0.274438,0.725562
2003-04,52859,189803,0.278494,0.721506
2004-05,56273,197626,0.284745,0.715255
2005-06,55792,194314,0.287123,0.712877
2006-07,57529,196072,0.293408,0.706592
2007-08,58885,200501,0.293689,0.706311
2008-09,59314,199054,0.297979,0.702021


In [93]:
colors = px.colors.qualitative.T10
fig = px.bar(missing_distance_info,
             x = missing_distance_info.index,
             y = [missing_distance_info["Present"],missing_distance_info["Missing"]],
             text_auto='.3f',
             color_discrete_sequence = colors,
             title = 'Shots with missing distance information per season'
             )
#fig.update_yaxes(range = [0,3.5])
fig.update_layout(yaxis_title="Percentage",xaxis={'type': 'category'})
fig.update_traces(textfont_size=20, textangle=0, textposition="outside", cliponaxis=False)
#missing_shot_info
fig.show()

In [97]:
per_season_stats = {}

#For 0-5
per_season_stats["[0-5)"] = get_percentage("[0-5)")

#For 5-10
per_season_stats["[5-10)"] = get_percentage("[5-10)")

#For 10-15
per_season_stats["[10-15)"] = get_percentage("[10-15)")

#For 15-20
per_season_stats["[15-20)"] = get_percentage("[15-20)")

#For 20-25
per_season_stats["[20-25)"] = get_percentage("[20-25)")

#For 25-30
per_season_stats["[25-30)"] = get_percentage("[25-30)")

#For 30-35
per_season_stats["[30-35)"] = get_percentage("[30-35)")

#For 35-40
per_season_stats["[35-40)"] = get_percentage("[35-40)")

#For 40-47
per_season_stats["[40-47)"] = get_percentage("[40-47)")

#For 47+
per_season_stats["[47+)"] = get_percentage("[47+)")


per_season_stats = pd.DataFrame(per_season_stats)

per_season_stats = per_season_stats.div(per_season_stats.sum(axis=1), axis=0)
per_season_stats


,[0-5),[5-10),[10-15),[15-20),[20-25),[25-30),[30-35),[35-40),[40-47),[47+)
season_name,,,,,,,,,,
2000-01,0.108230,0.168037,0.146715,0.280717,0.150293,0.139434,0.001772,0.001010,0.001952,0.001841
2001-02,0.089247,0.157876,0.147664,0.285992,0.159674,0.153050,0.001564,0.001097,0.001769,0.002067
2002-03,0.091795,0.159671,0.144361,0.280249,0.171738,0.145788,0.001327,0.000969,0.001779,0.002324
2003-04,0.098741,0.155560,0.138619,0.274097,0.174699,0.151704,0.001417,0.000971,0.001920,0.002271
2004-05,0.105474,0.142735,0.124935,0.276917,0.184057,0.158943,0.001471,0.001097,0.001953,0.002419
2005-06,0.110033,0.141552,0.113715,0.271293,0.188894,0.167345,0.001660,0.001011,0.001956,0.002541
2006-07,0.110543,0.140902,0.111409,0.262561,0.188137,0.179085,0.001718,0.001075,0.002050,0.002519
2007-08,0.095674,0.135952,0.116957,0.249739,0.201856,0.191483,0.001914,0.001299,0.002041,0.003086
2008-09,0.106798,0.137641,0.116416,0.249714,0.189380,0.191914,0.001997,0.001159,0.002004,0.002977


In [98]:
import plotly.express as px
colors = px.colors.qualitative.T10

# plotly
fig = px.bar(per_season_stats, 
             x = per_season_stats.index,
             y = [c for c in per_season_stats.columns],
             color_discrete_sequence = colors,
             title = 'Shooting attempts per season'
             )
#fig.update_yaxes(range = [0,3.5])
fig.update_layout(yaxis_title="Percentage",xaxis={'type': 'category'})

fig.show()